In [1]:
import pandas as pd
import numpy as np
import gc
import utils
import dataset
import seaborn as sns
import matplotlib.pyplot as plt
import analysis

In [2]:
app_train = dataset.get_data('../input/application_train.csv')

load data from ../input/application_train.csv - done in 3s


In [3]:
binary_features = [
    'FLAG_OWN_CAR',
    'FLAG_OWN_REALTY',
    
    'FLAG_DOCUMENT_2',
    'FLAG_DOCUMENT_3',
    'FLAG_DOCUMENT_4',
    'FLAG_DOCUMENT_5',
    'FLAG_DOCUMENT_6',
    'FLAG_DOCUMENT_7',
    'FLAG_DOCUMENT_8',
    'FLAG_DOCUMENT_9',
    'FLAG_DOCUMENT_10',
    'FLAG_DOCUMENT_11',
    'FLAG_DOCUMENT_12',
    'FLAG_DOCUMENT_13',
    'FLAG_DOCUMENT_14',
    'FLAG_DOCUMENT_15',
    'FLAG_DOCUMENT_16',
    'FLAG_DOCUMENT_17',
    'FLAG_DOCUMENT_18',
    'FLAG_DOCUMENT_19',
    'FLAG_DOCUMENT_20',
    'FLAG_DOCUMENT_21',
    'REG_REGION_NOT_LIVE_REGION',
    'REG_REGION_NOT_WORK_REGION',
    'LIVE_REGION_NOT_WORK_REGION',
    'REG_CITY_NOT_LIVE_CITY',
    'REG_CITY_NOT_WORK_CITY',
    'LIVE_CITY_NOT_WORK_CITY',
    'FLAG_MOBIL',
    'FLAG_EMP_PHONE',
    'FLAG_WORK_PHONE',
    'FLAG_CONT_MOBILE',
    'FLAG_PHONE',
    'FLAG_EMAIL',
    'CODE_GENDER',
    'EMERGENCYSTATE_MODE'
]

In [4]:
category_features = [
    'NAME_CONTRACT_TYPE',
    'NAME_EDUCATION_TYPE',
    'NAME_FAMILY_STATUS',
    'NAME_HOUSING_TYPE',
    'ORGANIZATION_TYPE',
    'WEEKDAY_APPR_PROCESS_START',
    'NAME_TYPE_SUITE',
    'NAME_INCOME_TYPE',
    'OCCUPATION_TYPE',
    'FONDKAPREMONT_MODE',
    'HOUSETYPE_MODE',
    'WALLSMATERIAL_MODE'
]

In [5]:
numerical_features = [_ for _ in app_train.columns if app_train[_].dtype == 'float64']
numerical_features = numerical_features + [
    'CNT_CHILDREN',
    'DAYS_BIRTH',
    'DAYS_EMPLOYED',
    'DAYS_ID_PUBLISH',
    'REGION_RATING_CLIENT_W_CITY',
    'REGION_RATING_CLIENT',
    'HOUR_APPR_PROCESS_START',
]

## 自动分析报告

In [6]:
analysis.generate_report(app_train, 'test.md', './pic', numerical_features, category_features + binary_features)

C:\SoftWare\Miniconda3\lib\site-packages\matplotlib\figure.py:1742: UserWarning: This figure includes Axes that are not compatible with tight_layout, so its results might be incorrect.
  warnings.warn("This figure includes Axes that are not "


Process AMT_INCOME_TOTAL - done in 4s
Process AMT_CREDIT - done in 3s
Process AMT_ANNUITY - done in 4s
Process AMT_GOODS_PRICE - done in 3s
Process REGION_POPULATION_RELATIVE - done in 3s
Process DAYS_REGISTRATION - done in 3s
Process OWN_CAR_AGE - done in 1s
Process CNT_FAM_MEMBERS - done in 4s
Process EXT_SOURCE_1 - done in 1s
Process EXT_SOURCE_2 - done in 3s
Process EXT_SOURCE_3 - done in 2s
Process APARTMENTS_AVG - done in 2s
Process BASEMENTAREA_AVG - done in 1s
Process YEARS_BEGINEXPLUATATION_AVG - done in 2s
Process YEARS_BUILD_AVG - done in 1s
Process COMMONAREA_AVG - done in 1s
Process ELEVATORS_AVG - done in 2s
Process ENTRANCES_AVG - done in 2s
Process FLOORSMAX_AVG - done in 2s


C:\SoftWare\Miniconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Process FLOORSMIN_AVG - done in 1s
Process LANDAREA_AVG - done in 1s
Process LIVINGAPARTMENTS_AVG - done in 1s
Process LIVINGAREA_AVG - done in 2s
Process NONLIVINGAPARTMENTS_AVG - done in 1s
Process NONLIVINGAREA_AVG - done in 2s
Process APARTMENTS_MODE - done in 2s
Process BASEMENTAREA_MODE - done in 1s
Process YEARS_BEGINEXPLUATATION_MODE - done in 2s
Process YEARS_BUILD_MODE - done in 1s
Process COMMONAREA_MODE - done in 1s
Process ELEVATORS_MODE - done in 2s
Process ENTRANCES_MODE - done in 2s
Process FLOORSMAX_MODE - done in 2s
Process FLOORSMIN_MODE - done in 1s
Process LANDAREA_MODE - done in 1s
Process LIVINGAPARTMENTS_MODE - done in 1s
Process LIVINGAREA_MODE - done in 2s
Process NONLIVINGAPARTMENTS_MODE - done in 1s
Process NONLIVINGAREA_MODE - done in 2s
Process APARTMENTS_MEDI - done in 2s
Process BASEMENTAREA_MEDI - done in 1s
Process YEARS_BEGINEXPLUATATION_MEDI - done in 2s
Process YEARS_BUILD_MEDI - done in 1s
Process COMMONAREA_MEDI - done in 1s
Process ELEVATORS_MEDI